# Unistore Latency Spikes Investigation

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Query data from the FDB server logs table
query = """
SELECT * FROM SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V LIMIT 100
"""
df = session.sql(query).to_pandas()

# Display basic info about the dataset
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# Show first few rows
print("\nFirst 5 rows:")
print(df.head())

# Display the dataframe in Streamlit if you want to visualize it
st.dataframe(df)


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
select
  time_slice(time, 300, 'second') as TIMEBUCKET,
  machine,
  sum(
      iff(
          Type = 'RedwoodMetrics',
          v:OpGetRange::float,
          null
      )
  ) as query_num,
  max(
      iff(
          Type = 'NormalReadLatencyMetrics',
          v:"P99"::float,
          null
      )
  ) as latency_p99,
  sum(iff(Type = 'StorageMetrics', split_part(v:FetchRangeReadBytes, ' ', 1)::float, null)) as fetch_bytes,
  sum(iff(Type = 'StorageMetrics', split_part(v:FetchRangeReadCount, ' ', 1)::float, null)) as fetch_count,
  sum(iff(Type = 'StorageMetrics', split_part(v:RowsQueried, ' ', 1)::float, null)) as rows_read,
  iff(
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    ) = 0,
    null,
    sum(iff(Type = 'StorageMetrics', split_part(v:FetchRangeReadBytes, ' ', 1)::float, null)) /
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    )
  ) as bytes_per_query,
    iff(
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    ) = 0,
    null,
    sum(iff(Type = 'StorageMetrics', split_part(v:RowsQueried, ' ', 1)::float, null)) /
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    )
  ) as rows_per_query
from
  SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
where
  Type in ('RedwoodMetrics', 'NormalReadLatencyMetrics', 'StorageMetrics')
  and log_group = 'prod1_autoprovision_88_100_fdbserver'
  and TIMEBUCKET between '2025-11-01 00:00:00'::timestamp_ntz
  and '2025-11-06 00:00:00'::timestamp_ntz
group by
  TIMEBUCKET, machine
having
  latency_p99 > 1
order by latency_p99 desc;

In [ ]:
WITH high_latency_events AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as latency_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'NormalReadLatencyMetrics'
        AND v:"P99"::float > 0.5
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
redwood_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:OpGetRange::float as op_get_range,
        COALESCE(v:OpTryNext::float, 0) + 
        COALESCE(v:OpNext::float, 0) + 
        COALESCE(v:OpNextBatch::float, 0) as op_nexts
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'RedwoodMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
storage_metrics AS (
    SELECT
        TIME,
        MACHINE,
        split_part(v:ClearRangeMutations, ' ', 1)::float as clear_range_mutations,
        split_part(v:GetRangeQueries, ' ', 1)::float as ss_get_range
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'StorageMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:ClearRangeMutations IS NOT NULL
)
SELECT
    h.TIME as high_latency_time,
    h.MACHINE,
    h.latency_p99,
    SUM(r.op_get_range) as total_op_get_range_1min_before,
    SUM(r.op_nexts) as total_op_nexts_1min_before,
    SUM(s.clear_range_mutations) as total_clear_range_mutations_1min_before,
    SUM(s.ss_get_range) as total_ss_get_range_1min_before
FROM
    high_latency_events h
    LEFT JOIN redwood_metrics r ON h.MACHINE = r.MACHINE
        AND r.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
    LEFT JOIN storage_metrics s ON h.MACHINE = s.MACHINE
        AND s.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
GROUP BY
    h.TIME,
    h.MACHINE,
    h.latency_p99
HAVING
    SUM(r.op_get_range) < 10000
    AND SUM(r.op_nexts) < 10000
    AND SUM(s.ss_get_range) > 100
ORDER BY
    h.TIME DESC;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")